<a href="https://colab.research.google.com/github/suryarasp/wsl2/blob/main/Nifty50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install necessary libraries
!pip install yfinance pandas

# Step 3: Download NSE 50 stocks data using yfinance
import yfinance as yf
import pandas as pd

# Define the ticker symbols for NSE 50 stocks
nse50_tickers = [
    'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS',
    'HINDUNILVR.NS', 'HDFC.NS', 'SBIN.NS', 'KOTAKBANK.NS', 'BHARTIARTL.NS',
    'ITC.NS', 'ASIANPAINT.NS', 'BAJFINANCE.NS', 'LT.NS', 'HCLTECH.NS',
    'AXISBANK.NS', 'MARUTI.NS', 'SUNPHARMA.NS', 'NESTLEIND.NS', 'ULTRACEMCO.NS',
    'TITAN.NS', 'M&M.NS', 'INDUSINDBK.NS', 'WIPRO.NS', 'POWERGRID.NS',
    'NTPC.NS', 'HDFCLIFE.NS', 'BAJAJFINSV.NS', 'TECHM.NS', 'JSWSTEEL.NS',
    'DIVISLAB.NS', 'TATASTEEL.NS', 'HEROMOTOCO.NS', 'BPCL.NS', 'ADANIENT.NS',
    'SHREECEM.NS', 'GRASIM.NS', 'BRITANNIA.NS', 'CIPLA.NS', 'ONGC.NS',
    'TATAMOTORS.NS', 'COALINDIA.NS', 'ADANIPORTS.NS', 'SBILIFE.NS', 'APOLLOHOSP.NS',
    'BAJAJ-AUTO.NS', 'EICHERMOT.NS', 'UPL.NS', 'HINDALCO.NS', 'DRREDDY.NS'
]
print(type(nse50_tickers))
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/input/ind_nifty200list.csv',index_col=False)
nse200_tickers= df.iloc[:, 0].tolist()
print(nse200_tickers)



In [ ]:
# Create a DataFrame to store the stock data
stock_data = []

# Fetch the data for each ticker
for ticker in nse200_tickers:
    stock = yf.Ticker(ticker)
    data = stock.history(period="5d")
    if len(data) < 2:
        continue
    closing_price = data['Close'].iloc[-1]
    prev_closing_price = data['Close'].iloc[-2]
    volume = data['Volume'].iloc[-1]
    prev_volume = data['Volume'].iloc[-2]
    volume_change = ((volume - prev_volume) / prev_volume) * 100
    price_change = ((closing_price - prev_closing_price) / prev_closing_price) * 100
    stock_data.append({
        'Symbol': ticker,
        'Closing Price': closing_price,
        'Prev Closing Price': prev_closing_price,
        'Volume': volume,
        'Prev Volume': prev_volume,
        'Volume Change (%)': volume_change,
        'Price Change (%)': price_change
    })

# Convert to DataFrame
df = pd.DataFrame(stock_data)

# Separate stocks with more than 5% volume change
significant_volume_change = df[df['Volume Change (%)'] > 5]

# Save the DataFrame to Google Drive
file_path = '/content/drive/My Drive/NSE50_Closing_Prices_Volume.csv'
df.to_csv(file_path, index=False)
print(f"File saved to {file_path}")

# Save the significant volume change DataFrame to Google Drive
significant_file_path = '/content/drive/My Drive/NSE50_Significant_Volume_Change.csv'
significant_volume_change.to_csv(significant_file_path, index=False)
print(f"Significant volume change file saved to {significant_file_path}")

# Load the CSV file from Google Drive
df_loaded = pd.read_csv(file_path)

# Display the loaded DataFrame
df_loaded

In [ ]:
import plotly.express as px
# Filter stocks with positive price change
positive_price_change = df[df['Price Change (%)'] > 0]
# Create a scatter plot
fig = px.scatter(
    positive_price_change,
    x='Volume Change (%)',
    y='Price Change (%)',
    size='Volume',
    color='Symbol',
    hover_name='Symbol',
    hover_data=['Closing Price', 'Prev Closing Price', 'Volume', 'Prev Volume'],
    title='Volume Change (%) vs. Price Change (%) for Positive Price Change Stocks',
    labels={
        'Volume Change (%)': 'Volume Change (%)',
        'Price Change (%)': 'Price Change (%)'
    }
)

# Show the plot
fig.show()

In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the ticker symbol
ticker_symbol = 'RVNL.NS'  # Example: Apple Inc.

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Get historical market data for the last month
historical_data = ticker.history(period='1mo')

# Extract required data (volume and closing price)
last_month_data = historical_data[['Volume', 'Close']]

# Create subplots with shared x-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=('Volume', 'Closing Price'))

# Adding volume trace to the first subplot
fig.add_trace(go.Bar(x=last_month_data.index, y=last_month_data['Volume'], name='Volume'),
              row=1, col=1)

# Adding closing price trace to the second subplot
fig.add_trace(go.Scatter(x=last_month_data.index, y=last_month_data['Close'], mode='lines', name='Closing Price'),
              row=2, col=1)

# Update axes titles
fig.update_xaxes(title_text='Date', row=2, col=1)
fig.update_yaxes(title_text='Volume', row=1, col=1)
fig.update_yaxes(title_text='Price', row=2, col=1)

# Update layout
fig.update_layout(title='Volume and Closing Price of {} - Last Month'.format(ticker_symbol),
                  height=600, width=800,
                  template='plotly_white')

# Show plot
fig.show()




# Select Net Income data
net_income_data = historical_data[['Net Income']]

# Calculate percentage growth in earnings quarter-to-quarter
net_income_data['Earnings Growth (%)'] = net_income_data['Net Income'].pct_change() * 100

# Filter to get data for the past 8 quarters (last 2 years)
net_income_data_last_8_quarters = net_income_data.tail(8)

# Display the earnings growth data as a table
print(net_income_data_last_8_quarters)